In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras import backend as K

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]

C:\Users\noumanschaudhry\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
#load data
df = pd.read_csv("creditcard_sample.csv")
print(df.Class.count())

113923


In [18]:
#assuming class =1 represents fraud as there are more entries in class 0 which is usually the normal case. count is corrent made sure by adding the two values
fraud = df[df.Class == 1]
print("Fraud: "+str(fraud.Class.count()))

normal = df[df.Class == 0]
print("Normal: "+str(normal.Class.count()))

Fraud: 242
Normal: 113681


In [5]:
#preprocessing data
from sklearn.preprocessing import StandardScaler

data = df.drop(['Time'], axis=1)

data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
X_train, X_test = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED)
X_train = X_train[X_train.Class == 0]
X_train = X_train.drop(['Class'], axis=1)

y_test = X_test['Class']
X_test = X_test.drop(['Class'], axis=1)

X_train = X_train.values
X_test = X_test.values

In [7]:
#NN model 

input_dim = X_train.shape[1]
encoding_dim = 14


#encoder
input_tensor = Input(shape=(input_dim, ))

encoderOut = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_tensor)
encoderOut = Dense(int(encoding_dim / 2), activation="relu")(encoderOut)

encoder = Model(input_tensor, encoderOut)


#decoder
decoder_input = Input(shape=(int(encoding_dim / 2),))
decoderOut = Dense(int(encoding_dim / 2), activation='tanh',name='decoder_input')(decoder_input)
decoderOut = Dense(input_dim, activation='relu',name='decoder_output')(decoderOut)

decoder = Model(decoder_input, decoderOut)

#autoencoder
autoInput = Input(shape=(input_dim, ))
encoderOut = encoder(autoInput)
decoderOut = decoder(encoderOut)
autoencoder = Model(inputs=autoInput, outputs=decoderOut)

print(input_dim)

30


In [8]:
#train
nb_epoch = 3
batch_size = 32

autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)

history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer]).history

Train on 90940 samples, validate on 22785 samples
Epoch 1/3
90940/90940 [==============================] - 18s 196us/step - loss: 901600544.8769 - acc: 1.0000 - val_loss: 897395963.0915 - val_acc: 1.0000
Epoch 2/3
32704/90940 [=========>....................] - ETA: 2s - loss: 907307269.3229 - acc: 1.0000

KeyboardInterrupt: 